# Exploring Data with Dataframes and Spark SQL
In this exercise, you will explore data using the Spark Dataframe API and Spark SQL.

## Load Data Using an Explicit Schema

Now you can load the data into a dataframe. If the structure of the data is known ahead of time, you can explicitly specify the schema for the dataframe.

Review the code in the cell below, which defines a schema for flight data before loading it from a text file. Then click the **&#9655;** button to the left of the cell to run it.

In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

flightSchema = StructType([
  StructField("DayofMonth", IntegerType(), False),
  StructField("DayOfWeek", IntegerType(), False),
  StructField("Carrier", StringType(), False),
  StructField("OriginAirportID", IntegerType(), False),
  StructField("DestAirportID", IntegerType(), False),
  StructField("DepDelay", IntegerType(), False),
  StructField("ArrDelay", IntegerType(), False),
])

flights = spark.read.csv('/data/raw-flight-data.csv', schema=flightSchema, header=True)
display(flights.limit(20))

## Infer a Data Schema
If the structure of the data source is unknown, you can have Spark automatically infer the schema.

In this case, you will load data about airports without knowing the schema.

Run the following cell to load airport data from a text file, inferring the column names and data types automatically.

In [ ]:
airports = spark.read.csv('/data/airports.csv', header=True, inferSchema=True)
display(airports.limit(20))

## Use Dataframe Methods
Spark DataFrames provide functions that you can use to extract and manipulate data. For example, you can use the **select** function to return a new dataframe containing columns selected from an existing dataframe.

In [ ]:
cities = airports.select("city", "name")
display(cities.limit(20))

## Combine Operations
You can combine functions in a single statement to perform multiple operations on a dataframe. In this case, you will use the **join** function to combine the **flights** and **airports** dataframes, and then use the **groupBy** and **count** functions to return the number of flights from each airport, and finally use the **orderBy** function to sort the results by the number of flights.

In [ ]:
flightsByOrigin = flights.join(airports, flights.OriginAirportID == airports.airport_id).groupBy("city").count().orderBy("count")
display(flightsByOrigin.limit(30))

## Count the Rows in a Dataframe
Now that you're familiar with working with dataframes, a key task when building predictive solutions is to explore the data, determing statistics that will help you understand the data before building predictive models. For example, how many rows of flight data do you actually have?

In [ ]:
flights.count()

## Determine the Presence of Duplicates
The data you have to work with won't always be perfect - often you'll want to *clean* the data; for example to detect and remove duplicates that might affect your model. You can use the **dropDuplicates** function to create a new dataframe with the duplicates removed, enabling you to determine how many rows are duplicates of other rows.

In [ ]:
flights.count() - flights.dropDuplicates().count()

## Identify Missing Values
As well as determining if duplicates exist in your data, you should detect missing values, and either remove rows containing missing data or replace the missing values with a suitable relacement. The **dropna** function creates a dataframe with any rows containing missing data removed - you can specify a subset of columns, and whether the row should be removed in *any* or *all* values are missing. You can then use this new dataframe to determine how many rows contain missing values.

In [ ]:
flights.count() - flights.dropDuplicates().dropna(how="any", subset=["ArrDelay", "DepDelay"]).count()

## Clean the Data
Now that you've identified that there are duplicates and missing values, you can clean the data by removing the duplicates and replacing the missing values. The **fillna** function replaces missing values with a specified replacement value. In this case, you'll remove all duplicate rows and replace missing **ArrDelay** and **DepDelay** values with **0**.

In [ ]:
data=flights.dropDuplicates().fillna(value=0, subset=["ArrDelay", "DepDelay"])
data.count()

## Explore the Data
Now that you've cleaned the data, you can start to explore it and perform some basic analysis. Let's start by examining the lateness of a flight. The dataset includes the **ArrDelay** field, which tells you how many minutes behind schedule a flight arrived. However, if a flight is only a few minutes behind schedule, you might not consider it *late*. Let's make our definition of lateness such that flights that arrive within 25 minutes of their scheduled arrival time are considered on-time, but any flights that are more than 25 minutes behind schedule are classified as *late*. We'll add a column to indicate this classification:

In [ ]:
data = data.select("DayofMonth", "DayOfWeek", "Carrier", "OriginAirportID","DestAirportID",
                   "DepDelay", "ArrDelay", ((col("ArrDelay") > 25).cast("Int").alias("Late")))
display(data.limit(20))

### Explore Summary Statistics and Data Distribution
Predictive modeling is based on statistics and probability, so we should take a look at the summary statistics for the columns in our data. The **describe** function returns a dataframe containing the **count**, **mean**, **standard deviation**, **minimum**, and **maximum** values for each numeric column.

In [ ]:
display(data.describe())

The *DayofMonth* is a value between 1 and 31, and the mean is around halfway between these values; which seems about right. The same is true for the *DayofWeek* which is a value between 1 and 7. *Carrier* is a string, so there are no numeric statistics; and we can ignore the statistics for the airport IDs - they're just unique identifiers for the airports, not actually numeric values. The departure and arrival delays range between 63 or 94 minutes ahead of schedule, and over 1,800 minutes behind schedule. The means are much closer to zero than this, and the standard deviation is quite large; so there's quite a bit of variance in the delays. The *Late* indicator is a 1 or a 0, but the mean is very close to 0; which implies that there significantly fewer late flights than non-late flights.

Let's verify that assumption by creating a table and using the **Spark SQL** API to run a SQL statement that counts the number of late and non-late flights:

In [ ]:
data.createOrReplaceTempView("flightData")
lateCounts = spark.sql("SELECT COUNT(*) AS Count, Late FROM flightData GROUP BY Late")
display(lateCounts)

Yes, it looks like there are significantly more non-late flights than late ones. You can see this more clearly with a visualization, so in the output above, select the **Chart** view to see the comparative counts as a bar chart..

Let's address the outliers and imbalanced classes in our data by removing rows with extreme delay values, and *undersampling* the more common on-time flights:

In [ ]:
from pyspark.sql.functions import rand

# Remove outliers - let's make the cut-off 150 minutes.
data = data.filter("DepDelay < 150 AND ArrDelay < 150")

# Separate the late and on-time flights
pos = data.filter("Late = 1")
neg = data.filter("Late = 0")

# undersample the most prevalent class to get a roughly even distribution
posCount = pos.count()
negCount = neg.count()
if posCount > negCount:
  pos = pos.sample(True, negCount/(negCount + posCount))
else:
  neg = neg.sample(True, posCount/(negCount + posCount))
  
# shuffle into random order (so a sample of the first 1000 has a mix of classes)
data = neg.union(pos).orderBy(rand())

# Replace the temporary table so we can query and visualize the balanced dataset
data.createOrReplaceTempView("flightData")

# Show the statistics
display(data.describe())

Now the maximums for the **DepDelay** and **ArrDelay** are clipped at under 150, and the mean value for the binary *Late* class is nearer 0.5; indicating a more or less even number of each class. We removed some data to accomplish this balancing act, but there are still a substantial number of rows for us to train a machine learning model with, and now the data is more balanced. Let's visualize the data again to confirm this:

In [ ]:
%%sql
SELECT Late, COUNT(*) AS Count FROM flightData GROUP BY Late

Display the data as a chart to compare the distribution of the **Late** classes as you did previously. There should now be a more even number of each class.

### Explore Relationships in the Data
Predictive modeling is largely based on statistical relationships between fields in the data. To design a good model, you need to understand how the data points relate to one another.

A common way to start exploring relationships is to create visualizations that compare two or more data values. For example, run the following query to compare arrival delay by carrier.


In [ ]:
%%sql
SELECT Carrier, ArrDelay FROM flightData

View the output as a chart, and then use the **View Options** button (which looks similar to **&#128463;<sub>*</sub>**) at the top-right of the output to configure the chart as follows:

- **Chart type**: Box plot
- **Key**: Carrier
- **Values**: ArrDelay

When you apply the view options, the box plots should should show that the median delay (the line in the middle of the box), and the distribution of delays varies by carrier; with some carriers having a higher median delay than others. The same is true for other features, such as the day of the week and the destination airport.



You may already suspect that there's likely to be a relationship between delarture delay and arrival delay, so let's examine that next. Run the next cell to retrieve the departure and arrival delays for each flight.

In [ ]:
%%sql
SELECT DepDelay, ArrDelay FROM flightData

View the utput as a chart, and set the view options as follows:

- **Chart type**: Scatter plot
- **Key**: DepDelay
- **Values**: ArrDelay
- **Series Group**: *blank*
- **Aggregation**: Avg

The scatter plot shows the average arrival deplay for each recorded departure delay. Note that the points form a diagonal line, which indicates a strong linear relationship between departure delay and arrival delay.

This linear relationship shows a *correlation* between these two values, which we can measure statistically. The **corr** function calculates a correlation value between -1 and 1, indicating the strength of correlation between two fields. A strong positive correlation (near 1) indicates that high values for one column are often found with high values for the other, which a strong negative correlation (near -1) indicates that *low* values for one column are often found with *high* values for the other. A correlation near 0 indicates little apparent relationship between the fields.

Run the following cell to see the correlation statistic for these two variables.

In [ ]:
data.corr("DepDelay", "ArrDelay")

The correlation is close to 1, indicating a reasonably strong positive correlation between departure delay and arrval delay. Flights that depart late, unsuprisingly, often arrive late!

In this notebook we've cleaned the flight data, and explored it to identify some potential relationships between features of the flights and their lateness.